In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
pip install git+https://github.com/huggingface/transformers.git accelerate==0.27.1 datasets>=2.16.1 bitsandbytes==0.41.3 peft==0.8.2 trl==0.7.10 wandb==0.16.3 huggingface_hub==0.20.3

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3sqyef2i


In [3]:
from datasets import load_dataset  # Hugging Face datasets package
mbpp = load_dataset("mbpp")  # train, validation, and test
humaneval = load_dataset("openai_humaneval")  # test only

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [15]:
tokenizer = AutoTokenizer.from_pretrained("TechxGenus/starcoder2-3b-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "TechxGenus/starcoder2-3b-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
def generate_output(prompts):
  output_texts = []
  for p in prompts:
    PROMPT = """### Instruction
    {instruction}
    ### Response
    """
    instruction = "Refactor the given Python program to a more readable, efficient, and maintainable one. You can assume that the given program is semantically correct. Do not change the external behavior of the program, and keep the syntactic and semantic correctness. Python programs should be in a code block. Do not explain anything in natural language. \n \n"
    instruction += p
    prompt = PROMPT.format(instruction=instruction)
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=2048)
    output_text = tokenizer.decode(outputs[0])
    output_dict = {'prompt': p, 'output': output_text}
    output_texts.append(output_dict)
    # print(tokenizer.decode(outputs[0]))
  return output_texts



In [25]:
import csv
def parse_results(data):
    results = []

    for output in data:
        prompt = output['prompt']
        output = output['output']
        result_dict = {'prompt': prompt,'output': output}
        results.append(result_dict)
    return results


def parsed_csv(file_path, results):
  keys = ['prompt', 'output']
  with open(file_path, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=keys)
      writer.writeheader()
      for result in results:
          writer.writerow(result)

In [24]:
humaneval_prompts = []
for i in humaneval["test"]:
  concat_prompt = i['prompt'] + i['canonical_solution']
  humaneval_prompts.append(concat_prompt)

humaneval_outputs = generate_output(humaneval_prompts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

NameError: name 'parse_results' is not defined

In [26]:
humaneval_results=parse_results(humaneval_outputs)
parsed_csv('/content/humaneval_parsed.csv', humaneval_results)

In [27]:
mbpp_prompts=[]
for i in mbpp["test"]:
  concat_prompt = i['code']
  mbpp_prompts.append(concat_prompt)

mbpp_outputs = generate_output(mbpp_prompts)
mbpp_results=parse_results(mbpp_outputs)
parsed_csv('/content/mbpp_parsed.csv', mbpp_results)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio